## Loading of Miller ECoG data of the joystick track task

includes some visualizations

In [1]:
#@title Data retrieval
import os, requests

fname = 'joystick_track.npz'
url = "https://osf.io/6jncm/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)


In [ ]:
#@title Install packages, import matplotlib and set defaults
# install packages to visualize brains and electrode locations
!pip install nilearn --quiet
!pip install nimare --quiet

from matplotlib import rcParams 
from matplotlib import pyplot as plt
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [3]:
#@title Data loading
import numpy as np

alldat = np.load(fname, allow_pickle=True)['dat']

# Select just one of the recordings here. This is subject 1, block 1.
dat = alldat[0]

print(dat.keys())


dict_keys(['V', 'targetX', 'targetY', 'cursorX', 'cursorY', 'locs'])


# Dataset info #

This is one of multiple ECoG datasets from Miller 2019, recorded in clinical settings with a variety of tasks. Raw data here:

https://exhibits.stanford.edu/data/catalog/zk881ps0522

`dat` contain 4 sessions from 4 subjects, and was used in these papers: 

*Schalk, G., et al. "Decoding two-dimensional movement trajectories using electrocorticographic signals in humans." Journal of neural engineering 4.3 (2007): 264.*

*Schalk, Gerwin, et al. "Two-dimensional movement control using electrocorticographic signals in humans." Journal of neural engineering 5.1 (2008): 75.*

From the dataset readme: 

*During the study, each patient was in a semi-recumbent position in a hospital bed about 1 m from a computer monitor. The patient used a joystick to maneuver a white cursor track a green target moving counter-clockwise in a circle of diameter 85% of monitor height ~1m away. The hand used to control the joystick was contralateral to the implanted electrode array.*

We also know that subject 0 was implanted in the left temporal lobe, while subject 2 was implanted in the right frontal lobe. 

Sample rate is always 1000Hz, and the ECoG data has been notch-filtered at 60, 120, 180, 240 and 250Hz, followed by z-scoring across the entire recording and conversion to float16 to minimize size. 

Variables are: 
* `dat['V']`: continuous voltage data (time by channels)
* `dat['targetX']`: position of the target on the screen
* `dat['targetY']`: position of the target on the screen
* `dat['cursorX']`: X position of the cursor controlled by the joystick 
* `dat['cursorY']`: X position of the cursor controlled by the joystick 
* `dat['locs`]: three-dimensional coordinates of the electrodes

In [ ]:
from nilearn import plotting  
from nimare import utils

plt.figure(figsize=(8,8))
locs = dat['locs']
view = plotting.view_markers(utils.tal2mni(locs), marker_labels  = ['%d'%k for k in np.arange(locs.shape[0])], marker_color = 'purple', marker_size=5)
view

In [ ]:
# compute correlations between voltage data and X/Y position of cursor
from scipy import signal
dat = alldat[3]

V = dat['V'].astype('float32')

nt, nchan = V.shape

targetX = dat['targetX'].flatten()
targetY = dat['targetY'].flatten()

cx = np.zeros(nchan,)
cy = np.zeros(nchan,)
for j in range(nchan):
  cx[j] = np.corrcoef(V[:,j], targetX)[0,1]
  cy[j] = np.corrcoef(V[:,j], targetY)[0,1]


In [ ]:
plt.subplot(1,4,1)
plt.plot(cx)
plt.plot(cy)

plt.ylabel('correlation with\n X / Y position of cursor')
plt.xlabel('channel index')


In [ ]:
# this one needs a lot more plots! 
# for some reason, I only see meaningful correlations in subjects 2 and 3, 
# but it's possible that there is spectral information that is more useful in those subjects